# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dudinka,69.4058,86.1778,294.05,54,93,0.90,RU,1691527554
1,1,mount pearl,47.5166,-52.7813,285.90,91,100,4.12,CA,1691482650
2,2,xifeng,42.7372,124.7222,297.38,76,27,3.36,CN,1691527214
3,3,albany,42.6001,-73.9662,293.47,82,100,0.45,US,1691488490
4,4,hawaiian paradise park,19.5933,-154.9731,300.88,74,20,4.12,US,1691510297


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    scale = 0.5
)

# Display the map
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df
city_data_df = city_data_df[((city_data_df["Humidity"] > 30) & (city_data_df["Humidity"] < 60))]
city_data_df = city_data_df[((city_data_df["Max Temp"] > 290) & (city_data_df["Max Temp"] < 305))]
city_data_df = city_data_df[((city_data_df["Cloudiness"] > 20) & (city_data_df["Cloudiness"] < 60))]
city_data_df = city_data_df[(city_data_df["Wind Speed"] < 2)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,75,kuanda,56.3161,116.0806,298.06,46,43,1.70,RU,1691526732
134,134,kasongo-lunda,-6.4667,16.8167,296.06,41,59,0.74,CD,1691557338
175,175,khandyga,62.6667,135.6000,295.40,38,24,1.05,RU,1691519927
381,381,salair,54.2312,85.7972,293.42,51,36,0.64,RU,1691534524
404,404,terra santa,-2.1042,-56.4869,300.00,59,53,0.93,BR,1691488238


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
75,kuanda,RU,56.3161,116.0806,46,
134,kasongo-lunda,CD,-6.4667,16.8167,41,
175,khandyga,RU,62.6667,135.6000,38,
381,salair,RU,54.2312,85.7972,51,
404,terra santa,BR,-2.1042,-56.4869,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey" : geoapify_key,
    "categories" : "accommodation.hotel"
    # 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude= row["Lng"]
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # # Make and API request using the params dictionaty
    # # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
        
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kuanda - nearest hotel: No hotel found
kasongo-lunda - nearest hotel: No hotel found
khandyga - nearest hotel: Апельсин
salair - nearest hotel: Лига
terra santa - nearest hotel: Hotel Novo Horizonte


,City,Country,Lat,Lng,Humidity,Hotel Name
75,kuanda,RU,56.3161,116.0806,46,No hotel found
134,kasongo-lunda,CD,-6.4667,16.8167,41,No hotel found
175,khandyga,RU,62.6667,135.6000,38,Апельсин
381,salair,RU,54.2312,85.7972,51,Лига
404,terra santa,BR,-2.1042,-56.4869,59,Hotel Novo Horizonte


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    scale = 0.5,
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)